# ML Model Development Problem 1  
**Title:** Bias Buster: Fair Loan Approval Classifier (Supervised Learning)

## Scenario
You’re building a binary classifier to **predict loan approvals**. The historical data has **embedded bias** across a protected attribute (`group` ∈ {A, B}).

## Objectives
- Data cleaning & feature engineering
- Model selection: try at least **2 algorithms** (e.g., Logistic Regression, Random Forest)
- Hyperparameter tuning (Grid/Random search ok)
- Validation: **cross-validation**
- Evaluation: **accuracy, precision, recall, F1**
- **Ethics:** compute **Demographic Parity Difference** and **Equal Opportunity Difference**; propose a mitigation (thresholding, reweighting, or feature handling) and document trade-offs (transparency, fairness, accountability, human agency).

## Deliverables
1. Working notebook that:
   - Trains ≥2 models and selects one final model with CV
   - Reports metrics listed above
   - Computes fairness metrics and applies a mitigation
   - Provides **Permutation Feature Importance** (XAI)
   - Produces a short **Model Card** (Markdown cell)
2. Export the final metrics summary as a printed Python dict.

## Scoring (auto-checked in notebook)
- Accuracy ≥ 0.78 and F1 ≥ 0.78 (20 pts)
- Equal Opportunity |Δ| ≤ 0.15 after mitigation (20 pts)
- Demographic Parity |Δ| ≤ 0.15 after mitigation (20 pts)
- Used CV + tuning + feature importance (20 pts)
- Model Card covering transparency/fairness/accountability/human-in-loop (20 pts)

**Total:** 100 pts

## Ethics Notes
- **Transparency**: log data cleaning steps and model choices
- **Explainability**: include feature importance + error analysis
- **Fairness**: measure parity and opportunity; mitigate and justify
- **Accountability & Human Agency**: define a manual review policy for borderline cases

---

**How to run**
- Open `AI_Quest_ML1_Bias_Buster.ipynb` and follow the numbered TODOs.
- Python 3.9+ recommended. Packages: `pandas`, `numpy`, `scikit-learn`, `matplotlib`.
- If needed, run the optional `pip install` cell inside the notebook.

In [ ]:
# Optional: install packages locally (uncomment if needed)
# !pip -q install pandas numpy scikit-learn matplotlib

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.inspection import permutation_importance

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Imports OK")

## 1) Data: Generate a synthetic, slightly biased dataset
We simulate historical approvals with a **latent bias**: group B has slightly lower base approval rate **independent** of true risk factors.

In [ ]:
# Generate synthetic data
n = 4000
income = np.random.lognormal(mean=10.5, sigma=0.5, size=n)  # ~ salaries
dti = np.clip(np.random.normal(0.3, 0.1, size=n), 0, 1)    # debt-to-income
credit = np.clip(np.random.normal(680, 70, size=n), 300, 850)
years = np.clip(np.random.exponential(scale=5, size=n), 0, 40)
age = np.clip(np.random.normal(38, 10, size=n), 18, 80)

# Protected attribute
group = np.random.choice(["A","B"], size=n, p=[0.55, 0.45])

# True approval probability (based on risk)
base = (
    0.35
    + 0.00003*(income)
    + 0.0008*(credit-600)
    + 0.02*(years>2)
    - 0.6*(dti>0.45)
)
# Inject historical bias: downward shift for group B
bias = np.where(group=="B", -0.06, 0.0)
p = np.clip(base + bias, 0.02, 0.98)

approved = (np.random.rand(n) < p).astype(int)

df = pd.DataFrame({
    "income": income,
    "dti": dti,
    "credit": credit,
    "years_employed": years,
    "age": age,
    "group": group,
    "approved": approved
})

# Add some missingness and noise
for col in ["income", "dti", "credit"]:
    idx = np.random.choice(n, size=int(0.03*n), replace=False)
    df.loc[idx, col] = np.nan

df.head()

### TODO 1 — Clean & engineer features (Transparency)
- Impute missing values (median/most_frequent)
- Scale numeric features (StandardScaler) — keep scaler for inference
- One-hot encode `group` for analysis but **do not** let leakage bias the model: consider both with/without `group` as a feature and justify your choice.

In [ ]:
# === YOUR CODE: cleaning & feature engineering ===
from sklearn.impute import SimpleImputer

num_cols = ["income","dti","credit","years_employed","age"]
cat_cols = ["group"]

# Example pipeline steps (you can change):
num_imputer = SimpleImputer(strategy="median")
X_num = num_imputer.fit_transform(df[num_cols])

X_num = pd.DataFrame(X_num, columns=num_cols, index=df.index)

# One-hot for group (for analysis)
X_cat = pd.get_dummies(df[cat_cols], drop_first=True)  # group_B as 1

X = pd.concat([X_num, X_cat], axis=1)
y = df["approved"].values
protected = (df["group"].values)  # keep for fairness

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[num_cols] = scaler.fit_transform(X[num_cols])

X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
    X_scaled, y, protected, test_size=0.25, random_state=RANDOM_SEED, stratify=y
)

print(X_train.shape, X_test.shape)

### TODO 2 — Baseline models + Cross-Validation
Train at least two models and perform CV + hyperparameter tuning.

In [ ]:
# === YOUR CODE: model selection & tuning ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

# Logistic Regression
logit = LogisticRegression(max_iter=200)
param_logit = {"C":[0.1,1.0,3.0]}
gs_logit = GridSearchCV(logit, param_logit, cv=cv, scoring="f1", n_jobs=-1)
gs_logit.fit(X_train, y_train)

# Random Forest
rf = RandomForestClassifier(random_state=RANDOM_SEED)
param_rf = {"n_estimators":[150,250], "max_depth":[None,8,12]}
gs_rf = GridSearchCV(rf, param_rf, cv=cv, scoring="f1", n_jobs=-1)
gs_rf.fit(X_train, y_train)

candidates = [("LogReg", gs_logit), ("RF", gs_rf)]
for name, gs in candidates:
    print(name, gs.best_params_, gs.best_score_)

# Pick a final model (you may change this selection logic)
final_name, final_gs = max(candidates, key=lambda t: t[1].best_score_)
final_model = final_gs.best_estimator_
final_model.fit(X_train, y_train)

print("Selected:", final_name)

### TODO 3 — Evaluation + Fairness metrics (before mitigation)
Compute accuracy, precision, recall, F1. Then compute:
- **Demographic Parity Difference** = P(ŷ=1|A) − P(ŷ=1|B)
- **Equal Opportunity Difference** = TPR_A − TPR_B

In [ ]:
def fairness_metrics(y_true, y_pred, prot):
    A = (prot=="A")
    B = (prot=="B")
    # Demographic parity
    pA = y_pred[A].mean() if A.any() else np.nan
    pB = y_pred[B].mean() if B.any() else np.nan
    dp = pA - pB
    # Equal opportunity (TPR)
    def tpr(mask):
        if (y_true[mask]==1).sum()==0: return np.nan
        return ((y_pred[mask]==1)&(y_true[mask]==1)).sum() / (y_true[mask]==1).sum()
    tprA = tpr(A)
    tprB = tpr(B)
    eod = tprA - tprB
    return {"demographic_parity_diff": dp, "equal_opportunity_diff": eod}

yhat_proba = final_model.predict_proba(X_test)[:,1] if hasattr(final_model,"predict_proba") else final_model.predict(X_test)
threshold = 0.5
yhat = (yhat_proba >= threshold).astype(int)

metrics_before = {
    "accuracy": accuracy_score(y_test, yhat),
    "precision": precision_score(y_test, yhat),
    "recall": recall_score(y_test, yhat),
    "f1": f1_score(y_test, yhat),
}
fair_before = fairness_metrics(y_test, yhat, prot_test)
print("Metrics BEFORE:", metrics_before)
print("Fairness BEFORE:", fair_before)

### TODO 4 — Mitigation
Try a simple mitigation and justify the trade-offs in a Markdown cell:
- **Option A:** Group-aware **thresholds** (choose threshold_A, threshold_B)
- **Option B:** **Re-weighting** during training (class_weight or sample_weight)
- **Option C:** Remove/retain `group` as feature; discuss proxy leakage and audits

Re-compute metrics after mitigation.

In [ ]:
# === Example mitigation: group-specific thresholds ===
th_A, th_B = 0.50, 0.46  # <-- you may tune

A = (prot_test=="A")
B = (prot_test=="B")
yhat_mitig = np.zeros_like(yhat)
yhat_mitig[A] = (yhat_proba[A] >= th_A).astype(int)
yhat_mitig[B] = (yhat_proba[B] >= th_B).astype(int)

metrics_after = {
    "accuracy": accuracy_score(y_test, yhat_mitig),
    "precision": precision_score(y_test, yhat_mitig),
    "recall": recall_score(y_test, yhat_mitig),
    "f1": f1_score(y_test, yhat_mitig),
}
fair_after = fairness_metrics(y_test, yhat_mitig, prot_test)
print("Metrics AFTER:", metrics_after)
print("Fairness AFTER:", fair_after)

### TODO 5 — Explainability (Permutation Feature Importance)

In [ ]:
if hasattr(final_model, "predict"):
    r = permutation_importance(final_model, X_test, y_test, scoring="f1", n_repeats=5, random_state=RANDOM_SEED)
    importances = pd.DataFrame({"feature": X_test.columns, "importance": r.importances_mean}).sort_values("importance", ascending=False)
    print(importances.head(10))
else:
    print("Permutation importance skipped (model lacks predict).")

### TODO 6 — Model Card (Markdown)
Document: data sources (synthetic), intended use, metrics, fairness results, mitigation chosen,
human-in-the-loop policy (manual review for borderline proba ∈ [0.45, 0.55]), and limitations.

In [ ]:
# === Auto-Scoring Cell ===
score = 0

acc_pass = metrics_after["accuracy"] >= 0.78
f1_pass = metrics_after["f1"] >= 0.78
if acc_pass and f1_pass: score += 20

eod_pass = abs(fair_after["equal_opportunity_diff"]) <= 0.15
if eod_pass: score += 20

dp_pass = abs(fair_after["demographic_parity_diff"]) <= 0.15
if dp_pass: score += 20

used_cv = True  # assume true if GridSearchCV above ran; feel free to toggle based on your pipeline
used_importance = True  # assume permutation importance executed
score += 20 if (used_cv and used_importance) else 0

# Model card existence check (simple heuristic): look for a markdown cell with "Model Card" in nb text manually.
model_card_claim = True  # set to True after you create it
if model_card_claim: score += 20

summary = {
    "accuracy_after": metrics_after["accuracy"],
    "f1_after": metrics_after["f1"],
    "equal_opportunity_diff_after": fair_after["equal_opportunity_diff"],
    "demographic_parity_diff_after": fair_after["demographic_parity_diff"],
    "score": score,
    "criteria": {
        "acc_f1": (acc_pass and f1_pass),
        "eod<=0.15": eod_pass,
        "dp<=0.15": dp_pass,
        "cv+importance": (used_cv and used_importance),
        "model_card": model_card_claim
    }
}

print("✅ Final Summary:", summary)